# Prepare

**NOTE:**
For the first iteration of your model, use only square feet of the home, number of bedrooms, and number of bathrooms to estimate the property's assessed value, taxvaluedollarcnt. You can expand this to other fields after you have completed an mvp (minimally viable product).

### Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

from acquire_kwame import get_zillow_data

In [2]:
df = get_zillow_data()
df.head(3)

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,latitude,longitude,regionidcounty,roomcnt,yearbuilt,taxvaluedollarcnt,assessmentyear,propertycountylandusecode,propertylandusetypeid
0,14297519,1727539,3.5,4.0,3.5,3100.0,6059.0,33634931.0,-117869207.0,1286.0,0.0,1998.0,1023282.0,2016.0,122,261.0
1,17052889,1387261,1.0,2.0,1.0,1465.0,6111.0,34449266.0,-119281531.0,2061.0,5.0,1967.0,464000.0,2016.0,1110,261.0
2,14186244,11677,2.0,3.0,2.0,1243.0,6059.0,33886168.0,-117823170.0,1286.0,6.0,1962.0,564778.0,2016.0,122,261.0


### Summarize the Data

In [34]:
print(f"The shape of the original dataframe:\n {df.shape[0]} rows and {df.shape[1]} columns.")

The shape of the original dataframe:
 77614 rows and 16 columns.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77614 entries, 0 to 77613
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77614 non-null  int64  
 1   id                            77614 non-null  int64  
 2   bathroomcnt                   77580 non-null  float64
 3   bedroomcnt                    77580 non-null  float64
 4   calculatedbathnbr             76964 non-null  float64
 5   calculatedfinishedsquarefeet  77379 non-null  float64
 6   fips                          77580 non-null  float64
 7   latitude                      77580 non-null  float64
 8   longitude                     77580 non-null  float64
 9   regionidcounty                77580 non-null  float64
 10  roomcnt                       77580 non-null  float64
 11  yearbuilt                     77310 non-null  float64
 12  taxvaluedollarcnt             77579 non-null  float64
 13  a

In [6]:
df.describe()

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,latitude,longitude,regionidcounty,roomcnt,yearbuilt,taxvaluedollarcnt,assessmentyear,propertylandusetypeid
count,7.761400e+04,7.761400e+04,77580.000000,77580.000000,76964.000000,77379.000000,77580.000000,7.758000e+04,7.758000e+04,77580.000000,77580.000000,77310.000000,7.757900e+04,77580.0,77580.000000
mean,1.300781e+07,1.496056e+06,2.298492,3.053222,2.316388,1784.925923,6048.813998,3.400837e+07,-1.182037e+08,2534.522235,1.476257,1968.611396,4.901476e+05,2016.0,261.824465
std,3.518694e+06,8.613448e+05,0.996726,1.140472,0.979684,954.247864,20.747162,2.652856e+05,3.594078e+05,801.445328,2.823673,23.793037,6.537942e+05,0.0,5.141564
min,1.071186e+07,3.490000e+02,0.000000,0.000000,1.000000,128.000000,6037.000000,3.333953e+07,-1.194754e+08,1286.000000,0.000000,1824.000000,1.000000e+03,2016.0,31.000000
25%,1.153821e+07,7.525952e+05,2.000000,2.000000,2.000000,1182.000000,6037.000000,3.381463e+07,-1.184150e+08,1286.000000,0.000000,1953.000000,2.068990e+05,2016.0,261.000000
50%,1.253006e+07,1.499186e+06,2.000000,3.000000,2.000000,1542.000000,6037.000000,3.402200e+07,-1.181810e+08,3101.000000,0.000000,1970.000000,3.588780e+05,2016.0,261.000000
75%,1.421100e+07,2.242084e+06,3.000000,4.000000,3.000000,2112.000000,6059.000000,3.417431e+07,-1.179286e+08,3101.000000,0.000000,1987.000000,5.690000e+05,2016.0,266.000000
max,1.676893e+08,2.985182e+06,18.000000,16.000000,18.000000,35640.000000,6111.000000,3.481877e+07,-1.175546e+08,3101.000000,15.000000,2016.000000,4.906124e+07,2016.0,275.000000


In [24]:
df.columns

Index(['parcelid', 'id', 'bathroomcnt', 'bedroomcnt', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'fips', 'latitude', 'longitude',
       'regionidcounty', 'roomcnt', 'yearbuilt', 'taxvaluedollarcnt',
       'assessmentyear', 'propertycountylandusecode', 'propertylandusetypeid'],
      dtype='object')

### Find and Handle Missing Values

In [7]:
df.isna().sum()

parcelid                          0
id                                0
bathroomcnt                      34
bedroomcnt                       34
calculatedbathnbr               650
calculatedfinishedsquarefeet    235
fips                             34
latitude                         34
longitude                        34
regionidcounty                   34
roomcnt                          34
yearbuilt                       304
taxvaluedollarcnt                35
assessmentyear                   34
propertycountylandusecode        34
propertylandusetypeid            34
dtype: int64

In [30]:
df.bathroomcnt.value_counts(dropna=False)

2.0     31576
3.0     17354
1.0     12945
2.5      6607
4.0      3356
1.5      1419
3.5      1036
5.0      1026
4.5       696
0.0       599
6.0       419
5.5       224
7.0       114
8.0       108
6.5        47
NaN        34
9.0        23
7.5        16
10.0        7
8.5         3
11.0        3
13.0        1
18.0        1
Name: bathroomcnt, dtype: int64

In [37]:
df.dropna()

,parcelid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,latitude,longitude,regionidcounty,roomcnt,yearbuilt,taxvaluedollarcnt,assessmentyear,propertycountylandusecode,propertylandusetypeid
0,14297519,1727539,3.5,4.0,3.5,3100.0,6059.0,33634931.0,-117869207.0,1286.0,0.0,1998.0,1023282.0,2016.0,122,261.0
1,17052889,1387261,1.0,2.0,1.0,1465.0,6111.0,34449266.0,-119281531.0,2061.0,5.0,1967.0,464000.0,2016.0,1110,261.0
2,14186244,11677,2.0,3.0,2.0,1243.0,6059.0,33886168.0,-117823170.0,1286.0,6.0,1962.0,564778.0,2016.0,122,261.0
3,12177905,2288172,3.0,4.0,3.0,2376.0,6037.0,34245180.0,-118240722.0,3101.0,0.0,1970.0,145143.0,2016.0,0101,261.0
4,10887214,1970746,3.0,3.0,3.0,1312.0,6037.0,34185120.0,-118414640.0,3101.0,0.0,1964.0,119407.0,2016.0,010C,266.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,11000655,673515,2.0,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,3101.0,0.0,1940.0,354621.0,2016.0,0100,261.0
77610,17239384,2968375,2.0,4.0,2.0,1612.0,6111.0,34300140.0,-118706327.0,2061.0,7.0,1964.0,67205.0,2016.0,1111,261.0
77611,12773139,1843709,1.0,3.0,1.0,1032.0,6037.0,34040895.0,-118038169.0,3101.0,0.0,1954.0,49546.0,2016.0,0100,261.0
77612,12826780,1187175,2.0,3.0,2.0,1762.0,6037.0,33937685.0,-117996709.0,3101.0,0.0,1955.0,522000.0,2016.0,0100,261.0
